## 1. Setting Up Media Pipe

In [5]:
import mediapipe as mp
import numpy as np
import pandas as pd
import cv2
import os
import warnings
warnings.filterwarnings('ignore')
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic


In [2]:
running_path = r'C:\Users\stanl\OneDrive\Desktop\Pose Estimation\second pose estimation model\running'
walking_path = r'C:\Users\stanl\OneDrive\Desktop\Pose Estimation\second pose estimation model\walking'

### Function Code to Extract Videos in Running Path

In [3]:
def get_videos(running_path):
    slices = [] #an empty list to store the values 
    #iterate through all the files and subdirectories
    for dirName, subdirList, fileList in os.walk(running_path):
        #iterate through all the files in the current directory
        for filename in fileList:
            #check if the file is mp4 video file
            if ".mp4" in filename.lower():
                slices.append(cv2.VideoCapture(os.path.join(dirName,filename),0))
    return slices #return the list of videos

In [4]:
video_paths = get_videos(running_path)

In [5]:
video_paths2 = get_videos(walking_path)

In [6]:
print(len(video_paths))
print(len(video_paths2))

12
13


In [28]:
# cap.shape

## 2. Capture Landmarks & Export to CSV

In [13]:
class_name = "walking"

In [14]:
# Function to get video file paths
def get_videos(running_path):
    video_paths = []
    for dirName, subdirList, fileList in os.walk(running_path):
        for filename in fileList:
            if ".mp4" in filename.lower():
                video_paths.append(os.path.join(dirName, filename))
    return video_paths

In [15]:
# List of video file paths
video_paths = get_videos(walking_path)

In [16]:
# setup mediapipe instance
with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for video_path in video_paths:
        # Initialize VideoCapture for the current video
        cap = cv2.VideoCapture(video_path)

        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                print(f"Finished processing {video_path}")
                break

            # Rest of your processing code remains the same
            #recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable=False
            
            #make detection
            results = holistic.process(image)
#             print(results.pose_landmarks)
            
            #recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            #1. Draw face landmarks
            mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
            
            #2. Right hand
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
            #3. Left hand
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
            
            #4. Pose Detection
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                         mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                         mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                         )
            

            # Export coordinates to a CSV file for each video
            try:
                # Extract Pose landmarks
                pose = results.pose_landmarks.landmark
                pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

                # Extract Face landmarks
                face = results.face_landmarks.landmark
                face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

                # Concatenate rows
                row = pose_row + face_row
#                 print(row)
                
                # Append class name
                row.insert(0, class_name)

                # Export to CSV for each video
#                 csv_filename = video_path.split('\\')[-1].replace('.mp4', '.csv')
#                 with open(csv_filename, mode='a', newline='') as f:
#                     csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                     csv_writer.writerow(row)
              
                #Export to one CSV file
                with open('walking.csv', mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"' , quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(row) 


            except:
                pass

        # Release video capture for the current video
        cap.release()
        cv2.destroyAllWindows()

Finished processing C:\Users\stanl\OneDrive\Desktop\Pose Estimation\second pose estimation model\walking\walking.mp4
Finished processing C:\Users\stanl\OneDrive\Desktop\Pose Estimation\second pose estimation model\walking\walking1.mp4
Finished processing C:\Users\stanl\OneDrive\Desktop\Pose Estimation\second pose estimation model\walking\walking10.mp4
Finished processing C:\Users\stanl\OneDrive\Desktop\Pose Estimation\second pose estimation model\walking\walking11.mp4
Finished processing C:\Users\stanl\OneDrive\Desktop\Pose Estimation\second pose estimation model\walking\walking12.mp4
Finished processing C:\Users\stanl\OneDrive\Desktop\Pose Estimation\second pose estimation model\walking\walking2.mp4
Finished processing C:\Users\stanl\OneDrive\Desktop\Pose Estimation\second pose estimation model\walking\walking3.mp4
Finished processing C:\Users\stanl\OneDrive\Desktop\Pose Estimation\second pose estimation model\walking\walking4.mp4
Finished processing C:\Users\stanl\OneDrive\Desktop\Po

In [18]:
print(results.face_landmarks)

None


## 3. Train Custom Model Using Scikit Learn

### 3.1 Read Collected Data and Process

In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [57]:
df = pd.read_csv('running.csv')
df1= pd.read_csv('walking.csv')

In [58]:
df_comb = pd.concat([df,df1])

In [59]:
df_comb.shape

(6322, 2005)

In [60]:
df_comb.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,running,0.485889,0.416545,-0.255803,0.996552,0.490618,0.396178,-0.255371,0.995555,0.495662,...,-0.004381,0,0.499212,0.390995,-0.007524,0,0.501148,0.388164,-0.008229,0
1,running,0.483980,0.424679,-0.412783,0.996895,0.490952,0.403882,-0.406051,0.996000,0.496653,...,-0.004751,0,0.498844,0.398006,-0.008303,0,0.500846,0.395468,-0.009027,0
2,running,0.482585,0.432175,-0.549736,0.997204,0.490683,0.412324,-0.532683,0.996397,0.496747,...,-0.004799,0,0.498499,0.406576,-0.008161,0,0.500440,0.404034,-0.008860,0
3,running,0.482328,0.446421,-0.516626,0.997399,0.490956,0.424717,-0.504155,0.996698,0.497583,...,-0.005017,0,0.498139,0.418235,-0.008500,0,0.500038,0.415614,-0.009160,0
4,running,0.482522,0.459509,-0.393758,0.997655,0.491890,0.436839,-0.380763,0.997024,0.498763,...,-0.005232,0,0.498840,0.431184,-0.008456,0,0.500725,0.428670,-0.009080,0


In [61]:
df_comb.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1741,walking,0.509722,0.491757,-0.209511,0.999979,0.512579,0.482341,-0.203010,0.999913,0.514687,...,-0.002407,0,0.515455,0.489354,-0.001972,0,0.515991,0.488139,-0.002047,0
1742,walking,0.509765,0.491793,-0.209933,0.999978,0.512511,0.482380,-0.203218,0.999906,0.514567,...,-0.002429,0,0.515184,0.489826,-0.002215,0,0.515720,0.488497,-0.002313,0
1743,walking,0.509374,0.492495,-0.218403,0.999976,0.512106,0.483315,-0.211904,0.999898,0.514176,...,-0.002528,0,0.515380,0.490905,-0.002240,0,0.515916,0.489593,-0.002331,0
1744,walking,0.509411,0.492968,-0.191145,0.999935,0.512187,0.483693,-0.183306,0.999763,0.514394,...,-0.002373,0,0.515291,0.491250,-0.002122,0,0.515851,0.489903,-0.002204,0
1745,walking,0.511094,0.495152,-0.141805,0.999928,0.512917,0.485698,-0.130074,0.999762,0.514781,...,-0.002432,0,0.515094,0.491683,-0.002168,0,0.515605,0.490389,-0.002256,0


In [62]:
df_comb[df_comb['class']=='running']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,running,0.485889,0.416545,-0.255803,0.996552,0.490618,0.396178,-0.255371,0.995555,0.495662,...,-0.004381,0,0.499212,0.390995,-0.007524,0,0.501148,0.388164,-0.008229,0
1,running,0.483980,0.424679,-0.412783,0.996895,0.490952,0.403882,-0.406051,0.996000,0.496653,...,-0.004751,0,0.498844,0.398006,-0.008303,0,0.500846,0.395468,-0.009027,0
2,running,0.482585,0.432175,-0.549736,0.997204,0.490683,0.412324,-0.532683,0.996397,0.496747,...,-0.004799,0,0.498499,0.406576,-0.008161,0,0.500440,0.404034,-0.008860,0
3,running,0.482328,0.446421,-0.516626,0.997399,0.490956,0.424717,-0.504155,0.996698,0.497583,...,-0.005017,0,0.498139,0.418235,-0.008500,0,0.500038,0.415614,-0.009160,0
4,running,0.482522,0.459509,-0.393758,0.997655,0.491890,0.436839,-0.380763,0.997024,0.498763,...,-0.005232,0,0.498840,0.431184,-0.008456,0,0.500725,0.428670,-0.009080,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,running,0.806271,0.164569,-2.471396,0.988067,0.855412,0.120868,-2.383683,0.988633,0.888565,...,-0.020782,0,0.903381,0.115992,0.016878,0,0.911564,0.113799,0.017792,0
4572,running,0.871545,0.183266,-2.785464,0.988725,0.918004,0.138386,-2.684727,0.989112,0.951719,...,-0.018584,0,0.976474,0.142280,0.030548,0,0.985671,0.138561,0.032290,0
4573,running,0.884755,0.195110,-3.084990,0.989335,0.932358,0.149402,-2.989568,0.989484,0.968043,...,-0.015453,0,0.976166,0.143064,0.032876,0,0.985002,0.140176,0.034526,0
4574,running,0.965964,0.218372,-2.181465,0.989744,1.009377,0.171995,-2.029280,0.989986,1.044720,...,-0.005076,0,1.066503,0.167336,0.064728,0,1.075040,0.162877,0.068282,0


In [63]:
df_comb[df_comb['class']=='walking']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,walking,0.739667,0.344423,-0.012661,0.999998,0.735189,0.335232,0.008083,0.999997,0.734425,...,0.003549,0,0.747232,0.338181,0.018280,0,0.747316,0.337497,0.019400,0
1,walking,0.739636,0.344414,-0.032703,0.999998,0.735183,0.335303,-0.013794,0.999997,0.734446,...,0.003040,0,0.745682,0.339299,0.016878,0,0.745749,0.338598,0.017953,0
2,walking,0.739575,0.345800,0.040964,0.999998,0.735128,0.336988,0.066227,0.999997,0.734438,...,0.006265,0,0.743964,0.342001,0.021203,0,0.743988,0.341338,0.022307,0
3,walking,0.739377,0.350469,-0.012318,0.999998,0.735127,0.340846,0.010717,0.999996,0.734441,...,0.003759,0,0.744457,0.342062,0.018220,0,0.744606,0.341393,0.019328,0
4,walking,0.740601,0.354311,-0.017117,0.999996,0.736014,0.344418,0.008009,0.999993,0.735214,...,0.003686,0,0.742289,0.343982,0.017894,0,0.742207,0.343290,0.019017,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,walking,0.509722,0.491757,-0.209511,0.999979,0.512579,0.482341,-0.203010,0.999913,0.514687,...,-0.002407,0,0.515455,0.489354,-0.001972,0,0.515991,0.488139,-0.002047,0
1742,walking,0.509765,0.491793,-0.209933,0.999978,0.512511,0.482380,-0.203218,0.999906,0.514567,...,-0.002429,0,0.515184,0.489826,-0.002215,0,0.515720,0.488497,-0.002313,0
1743,walking,0.509374,0.492495,-0.218403,0.999976,0.512106,0.483315,-0.211904,0.999898,0.514176,...,-0.002528,0,0.515380,0.490905,-0.002240,0,0.515916,0.489593,-0.002331,0
1744,walking,0.509411,0.492968,-0.191145,0.999935,0.512187,0.483693,-0.183306,0.999763,0.514394,...,-0.002373,0,0.515291,0.491250,-0.002122,0,0.515851,0.489903,-0.002204,0


In [64]:
X = df_comb.drop('class',axis=1) #selecting suitable x variables
y = df_comb['class'] # target values


In [65]:
y.head()

0    running
1    running
2    running
3    running
4    running
Name: class, dtype: object

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1000)

In [67]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4425, 2004), (1897, 2004), (4425,), (1897,))

### 3.2 Train Machine Learning Classification Model

In [68]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [69]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [70]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [71]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [72]:
fit_models['rc'].predict(X_test)

array(['running', 'running', 'running', ..., 'walking', 'running',
       'running'], dtype='<U7')

### 3.3 Evaluate and Serialize the Model

In [73]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle

In [74]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print("The accuracy score of the different ML algorithms: ",algo, accuracy_score(y_test, yhat))
    print(algo, classification_report(y_test,yhat))

The accuracy score of the different ML algorithms:  lr 0.996309963099631
lr               precision    recall  f1-score   support

     running       1.00      1.00      1.00      1384
     walking       0.99      0.99      0.99       513

    accuracy                           1.00      1897
   macro avg       1.00      1.00      1.00      1897
weighted avg       1.00      1.00      1.00      1897

The accuracy score of the different ML algorithms:  rc 0.9920927780706379
rc               precision    recall  f1-score   support

     running       1.00      0.99      0.99      1384
     walking       0.98      0.99      0.99       513

    accuracy                           0.99      1897
   macro avg       0.99      0.99      0.99      1897
weighted avg       0.99      0.99      0.99      1897

The accuracy score of the different ML algorithms:  rf 0.9994728518713759
rf               precision    recall  f1-score   support

     running       1.00      1.00      1.00      1384
     wa

In [75]:
y_test

3901    running
585     running
3470    running
2659    running
1165    walking
         ...   
687     walking
557     walking
1463    walking
1823    running
2517    running
Name: class, Length: 1897, dtype: object

In [76]:
with open('second_model.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)

## 4. Make Detections/Predictions with the Model

In [2]:
with open('second_model.pkl','rb') as f:
    model= pickle.load(f)

<IPython.core.display.Javascript object>

In [3]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [9]:
import warnings
warnings.filterwarnings('ignore')

cap = cv2.VideoCapture('C:/Users/stanl/OneDrive/Desktop/Pose Estimation/second pose estimation model/running.mp4')
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("")
            break
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

walking [0.24 0.76]
walking [0.23 0.77]
walking [0.13 0.87]
walking [0.13 0.87]
walking [0.14 0.86]
walking [0.14 0.86]
walking [0.13 0.87]
walking [0.27 0.73]
walking [0.4 0.6]
walking [0.42 0.58]
walking [0.43 0.57]
walking [0.43 0.57]
walking [0.43 0.57]
walking [0.42 0.58]
walking [0.23 0.77]
walking [0.23 0.77]
walking [0.24 0.76]
walking [0.27 0.73]
walking [0.22 0.78]
walking [0.22 0.78]
walking [0.24 0.76]
walking [0.23 0.77]
walking [0.23 0.77]
walking [0.29 0.71]
walking [0.45 0.55]
walking [0.48 0.52]
walking [0.49 0.51]
running [0.53 0.47]
running [0.54 0.46]
running [0.67 0.33]
running [0.82 0.18]
running [0.94 0.06]
running [0.92 0.08]
running [0.92 0.08]
running [0.92 0.08]
running [0.88 0.12]
running [0.84 0.16]
running [0.9 0.1]
running [0.89 0.11]
running [0.94 0.06]
running [0.92 0.08]
running [0.93 0.07]
running [0.92 0.08]
running [0.92 0.08]
running [0.94 0.06]
running [0.95 0.05]
running [0.9 0.1]

